In [1]:
import numpy as np
import pandas as pd
import torch
import tensorly as tl
# library for TTD   (Tensorly  (search))

from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tensorly.decomposition import tensor_train
from tensorly.decomposition import tensor_train
# different forms of tensor
from sklearn.preprocessing import MinMaxScaler
from tensorly import tt_to_tensor
from tensorly.decomposition import matrix_product_state
# tensor operations


#from math import ceil
#from google.colab import drive
#drive.mount('/content/gdrive')
import torch.nn.init as init
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader
import time
import os
import copy
import torch.nn.functional as Fun
print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)
print("GPU is available?", torch.cuda.is_available())

PyTorch Version: 1.13.1
Torchvision Version: 0.14.1
GPU is available? True


In [2]:
dtype = torch.float
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
features = list()
with open("/home/c/cl237/TenBCD/UCI HAR/data/features.txt") as f:
    features = [line.split()[1] for line in f.readlines()]
print('# of Features: {}'.format(len(features)))

# of Features: 561


## get train data

In [4]:
pd_X_train = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/X_train.txt', delim_whitespace=True, header=None)
pd_X_train.columns = features
pd_y_train = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/y_train.txt', delim_whitespace=True, names=['Activity'])

N = len(pd_X_train)

K = 6

x = pd_X_train.values
y = pd_y_train.values

X_train=torch.tensor(x,dtype = torch.float32,device=device)
y_train=torch.tensor(y,dtype = int) #dtype=torch.int,device=device)

y_train = torch.flatten(y_train)

X_train = torch.t(X_train)

y_one_hot = torch.zeros(N, K).scatter_(1, torch.reshape(y_train-1, (N, 1)), 1)
y_one_hot = torch.t(y_one_hot).to(device=device)
y_train = y_train.to(device=device)

## get test data

In [5]:
pd_X_test = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/X_test.txt', delim_whitespace=True, header=None)
pd_X_test.columns = features
pd_y_test = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/y_test.txt', delim_whitespace=True, names=['Activity'])

N_test = len(pd_X_test)

x = pd_X_test.values
y = pd_y_test.values

X_test=torch.tensor(x,dtype = torch.float32, device=device)
y_test=torch.tensor(y,dtype = int)

y_test = torch.flatten(y_test)

X_test = torch.t(X_test)

y_test_one_hot = torch.zeros(N_test, K).scatter_(1, torch.reshape(y_test-1, (N_test, 1)), 1)
y_test_one_hot = torch.t(y_test_one_hot).to(device=device)
y_test = y_test.to(device=device)

In [6]:
def updateV(U1,U2,W,b,rho,gamma): 
    _, d = W.size()
    I = torch.eye(d, device=device) #Returns a 2-D tensor with ones on the diagonal and zeros elsewhere.
    U1 = nn.ReLU()(U1)
    _, col_U2 = U2.size()
    Vstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(W),W))+gamma*I), rho*torch.mm(torch.t(W),U2-b.repeat(1,col_U2))+gamma*U1)
    # check how to get V* W*
    # torch.mm*=(input,mat2) performs a matrix multiplication of the matrices input and mat2.
    #   if input is a (n*m) tensor,mat2 is a (m*p) tensor, output will be (n*p) tensor
    return Vstar

def updateWb_org(U, V, W, b, alpha, rho): 
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse(alpha*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

def updateWb(U, V, W, b, W_tensor_rec, alpha, rho,tau): 
    W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
    W_tensor2matrix = W_tensor_rec.reshape(W.shape)
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+tau*W_tensor2matrix+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse((alpha+tau)*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

# U update  E(1) in global convergent
def relu_prox(a, b, gamma, d, N):
    val = torch.empty(d,N, device=device)
    x = (a+gamma*b)/(1+gamma)
    y = torch.min(b,torch.zeros(d,N, device=device))

    val = torch.where(a+gamma*b < 0, y, torch.zeros(d,N, device=device))
    val = torch.where(((a+gamma*b >= 0) & (b >=0)) | ((a*(gamma-np.sqrt(gamma*(gamma+1))) <= gamma*b) & (b < 0)), x, val)
    val = torch.where((-a <= gamma*b) & (gamma*b <= a*(gamma-np.sqrt(gamma*(gamma+1)))), b, val)
    return val

In [7]:
########### parameter setup
#df = pd.DataFrame()
#df.to_csv('C:/Users/Mark/Desktop/rank40_100times.csv')
niter = 500
rank = 80
tau = 5
alpha = 1
gamma = 0.5
rho = 0.5

#alpha1 = alpha2 = alpha3 = alpha4 = alpha5 = alpha6 = alpha7 = alpha8 = alpha9 = alpha10 = alpha
#for rank in (35,): #(25,30,35)(100,180,220,260,300,340,380)(20,60,100,140,180,220,260,300,340,380)
#    for tau in (400,500): #(300,400,500)(10,50,100,200,300)(10,50,100,200,300)
#        for gamma in (0.5,0.8,2): #(0.5,0.8,2)(0.5,0.8)(0.5,1,1.5,2,3)
            #gamma1 = gamma2 = gamma3 = gamma4 = gamma5 = gamma
#            for rho in (0.5,0.8,2): #(0.5,0.8)(1,2)
                #rho1 = rho2 = rho3 = rho4 = rho5= rho
#                for alpha in (0.5,1,1.5,2):
#                    print ("rank=",rank, "tau=",tau, "gamma=",gamma, "rho=",rho, "alpha",alpha)
                    #print('Compression Ratio', ((1024*28*28+10*1024+(8*(rank)+32*np.square(rank))*2)/(1024*28*28+10*1024+1024*1024*2)), (8*(rank)+32*np.square(rank))*2/(1024*1024*2))
        
print ("rank=",rank, "tau=",tau, "gamma=",gamma, "rho=",rho, "alpha",alpha)

loss1 = np.empty(niter)
loss2 = np.empty(niter)
accuracy_train = np.empty(niter)
accuracy_test = np.empty(niter)
time1 = np.empty(niter)

results = torch.zeros(1, 5, niter)



for Out_iter in range(1):
    rank_initial = 400
    seed = 10 + 10*Out_iter
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)

    d0 = 561 #561 =3*11*17
    d1 =  1024    # 2^10
    d2 =  1024  
    d3 = 1024
    d4 = 512      # 2^9
    d5 = 6 

    W1 = 0.2*init.xavier_uniform_(torch.empty(d1, d0, device=device), gain=1.0)
    #W1 = 0.01*torch.randn(d1, d0, device=device)
    W1_torch_tensor = W1.reshape((6,22,34,2,2,2,2,2,2,2))
    W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
    factors1 = tensor_train(W1_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W1_tl_tensor_rec = tt_to_tensor(factors1)
    b1 = 0*torch.ones(d1, 1, device=device) # 0 is stable


    W2 = 0.2*init.xavier_uniform_(torch.empty(d2, d1, device=device), gain=1.0)
    W2_torch_tensor = W2.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy())
    factors2 = tensor_train(W2_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    # we have 9 rank_initial (r1,...,r9) here plus 2 of 1s (r0 & r10).
    # factors: set of tensor cores
    # http://tensorly.org/stable/user_guide/tensor_decomposition.html
    # http://tensorly.org/stable/modules/generated/tensorly.decomposition.tensor_train.html#tensorly.decomposition.tensor_train
    W2_tl_tensor_rec = tt_to_tensor(factors2)
    # tt_to_tensor(factors):Re-assembles ‘factors’, which represent a tensor in TT format into the corresponding full tensor
    #      facros: list of 3d-arrays tt-cores           output_tensor: ndarray      
    b2 = 0*torch.ones(d2, 1, device=device)


    W3 = 0.2*init.xavier_uniform_(torch.empty(d3, d2, device=device), gain=1.0)
    W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
    factors3 = tensor_train(W3_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W3_tl_tensor_rec = tt_to_tensor(factors3)
    b3 = 0*torch.ones(d3, 1, device=device)

    W4 = 0.2*init.xavier_uniform_(torch.empty(d4, d3, device=device), gain=1.0)
    W4_torch_tensor = W4.reshape((4,4,4,4,4,4,4,4,4,2)) # 8 number of 4s, 2 number of 8s
    W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())
    factors4 = tensor_train(W4_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W4_tl_tensor_rec = tt_to_tensor(factors4)
    b4 = 0*torch.ones(d4, 1, device=device)


    W5 = 0.2*init.xavier_uniform_(torch.empty(d5, d4, device=device), gain=1.0)
    b5 = 0*torch.ones(d5, 1, device=device)

    # W4 = 0.01*torch.randn(d4, d3, device=device)
    # W4_torch_tensor = W4.reshape((50,30,K))
    # W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())
    # core4, tucker_factors4 = tucker(W4_tl_tensor, rank=(40,28,K), init='svd', tol=10e-5, random_state=12345)
    # W4_tl_tensor_rec = tl.tucker_to_tensor((core4, tucker_factors4))
    # b4 = 0.1*torch.ones(d4, 1, device=device)

    U1 = torch.addmm(b1.repeat(1, N), W1, X_train)
    V1 = nn.ReLU()(U1)
    U2 = torch.addmm(b2.repeat(1, N), W2, V1)
    V2 = nn.ReLU()(U2)
    U3 = torch.addmm(b3.repeat(1, N), W3, V2)
    V3 = nn.ReLU()(U3)
    U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    V4 = nn.ReLU()(U4)
    U5 = torch.addmm(b5.repeat(1, N), W5, V4)
    V5 = U5 
    # U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    # V4 = U4



    # Iterations
    print('Train on', N, 'samples, validate on', N_test, 'samples')
    for k in range(niter):
        start = time.time()

 # update for last layer
        # update V4
        V5 = (y_one_hot + gamma*U5 + alpha*V5)/(1 + gamma + alpha)

        # update U4 
        U5 = (gamma*V5 + rho*(torch.mm(W5,V4) + b5.repeat(1,N)))/(gamma + rho)

        # update W4 and b4
        W5, b5 = updateWb_org(U5,V4,W5,b5,alpha,rho)
        # W3_torch_tensor = W3.reshape((d3,4,4,4,4,4,4))
        # W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
        # factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,1))
        # W3_tl_tensor_rec = tt_to_tensor(factors3)



  # update for 4th layer
        # update V3
        V4 = updateV(U4,U5,W5,b5,rho,gamma)

        # update U3
        U4 = relu_prox(V4,(rho*torch.addmm(b4.repeat(1,N), W4, V3) + alpha*U4)/(rho + alpha),(rho + alpha)/gamma,d4,N)

        # update W3 and b3
        W4, b4 = updateWb(U4,V3,W4,b4,W4_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W4_torch_tensor = W4.reshape((4,4,4,4,4,4,4,4,4,2))
        W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors4 = tensor_train(W4_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W4_tl_tensor_rec = tt_to_tensor(factors4)

  # update for 3nd layer
        # update V3
        V3 = updateV(U3,U4,W4,b4,rho,gamma)

        # update U3
        U3 = relu_prox(V3,(rho*torch.addmm(b3.repeat(1,N), W3, V2) + alpha*U3)/(rho + alpha),(rho + alpha)/gamma,d3,N)

        # update W3 and b3
        W3, b3 = updateWb(U3,V2,W3,b3,W3_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4))
        W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W3_tl_tensor_rec = tt_to_tensor(factors3)

 # update for 2nd layer
        # update V2
        V2 = updateV(U2,U3,W3,b3,rho,gamma)

        # update U2
        U2 = relu_prox(V2,(rho*torch.addmm(b2.repeat(1,N), W2, V1) + alpha*U2)/(rho + alpha),(rho + alpha)/gamma,d2,N)

        # update W2 and b2
        W2, b2 = updateWb(U2,V1,W2,b2,W2_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W2_torch_tensor = W2.reshape((4,4,4,4,4,4,4,4,4,4))
        W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy()) 
        factors2 = tensor_train(W2_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W2_tl_tensor_rec = tt_to_tensor(factors2)


# update for 1st layer
        # update V1
        V1 = updateV(U1,U2,W2,b2,rho,gamma)

        # update U1
        U1 = relu_prox(V1,(rho*torch.addmm(b1.repeat(1,N), W1, X_train) + alpha*U1)/(rho + alpha),(rho + alpha)/gamma,d1,N)

        # update W1 and b1
        W1, b1 = updateWb(U1,X_train,W1,b1,W1_tl_tensor_rec, alpha,rho,tau)

        # G update
        W1_torch_tensor = W1.reshape((6,22,34,2,2,2,2,2,2,2))
        W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
        factors1 = tensor_train(W1_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W1_tl_tensor_rec = tt_to_tensor(factors1)




 # prediction for trainning data
        a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), X_train))
        #a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_train))
        a4_train = nn.ReLU()(torch.addmm(b4.repeat(1, N), torch.as_tensor(W4_tl_tensor_rec,device=device).reshape((d4, d3)).float(), a3_train))
        #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        pred = torch.argmax(torch.addmm(b5.repeat(1, N), W5, a4_train), dim=0)
        # check argmax and addmm, dim=0

 #Prediction for test data
        a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), X_test))
        #a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_test))
        a4_test = nn.ReLU()(torch.addmm(b4.repeat(1, N_test), torch.as_tensor(W4_tl_tensor_rec,device=device).reshape((d4, d3)).float(), a3_test))
        pred_test = torch.argmax(torch.addmm(b5.repeat(1, N_test), W5, a4_test), dim=0)

        # a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), x_train))
        # a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        # # a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_train))
        # #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        # pred = torch.argmax(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_train), dim=0)

        # a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), x_test))
        # a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        # # a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_test))
        # pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_test), dim=0)

        # a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        # a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), W2, a1_train))

        # #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        # pred = torch.argmax(torch.addmm(b3.repeat(1, N), W3, a2_train), dim=0)

        # a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        # a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), W2, a1_test))
        # pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), W3, a2_test), dim=0)




   #emperical loss
        loss1[k] = gamma/2*torch.pow(torch.dist(V5,y_one_hot,2),2).cpu().numpy()
        # torch.pow:Takes the power of each element in input with exponent and returns a tensor with the result.
        # torch.disk: Returns the p-norm of (input - other)
        # Eq (5) in paper
        loss2[k] = loss1[k] + rho/2*torch.pow(torch.dist(torch.addmm(b1.repeat(1,N), W1, X_train),U1,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b2.repeat(1,N), W2, V1),U2,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b3.repeat(1,N), W3, V2),U3,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b4.repeat(1,N), W4, V3),U4,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b5.repeat(1,N), W5, V4),U5,2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V1,nn.ReLU()(U1),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V2,nn.ReLU()(U2),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V3,nn.ReLU()(U3),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V4,nn.ReLU()(U4),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V5,U5,2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W1.reshape((6,22,34,2,2,2,2,2,2,2)),torch.as_tensor(W1_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W2.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W2_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W3.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W4.reshape((4,4,4,4,4,4,4,4,4,2)),torch.as_tensor(W4_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \

        # +tau/2*torch.pow(torch.dist(W3.reshape((d3,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() 
        # +tau/2*torch.pow(torch.dist(W4.reshape((50,30,K)),torch.as_tensor(W4_tl_tensor_rec,device=device),2),2).cpu().numpy() 

        # compute training accuracy
        correct_train = pred == y_train-1
        accuracy_train[k] = np.mean(correct_train.cpu().numpy())

        # compute validation accuracy
        correct_test = pred_test == y_test-1
        accuracy_test[k] = np.mean(correct_test.cpu().numpy())

        # compute training time
        stop = time.time()
        duration = stop - start
        time1[k] = duration

        # print results
        print('Repeatition', Out_iter + 1, 'Epoch', k + 1, '/', niter, '\n', 
              '-', 'time:', time1[k], '-', 'sq_loss:', loss1[k], '-', 'tot_loss:', loss2[k], 
              '-', 'acc:', accuracy_train[k], '-', 'val_acc:', accuracy_test[k])

##############
############## compute CR
    factors1_shape=[f.shape for f in factors1]
    Sum_of_variables_factors1=sum(list(x*y*z for x,y,z in factors1_shape))
    factors2_shape=[f.shape for f in factors2]
    Sum_of_variables_factors2=sum(list(x*y*z for x,y,z in factors2_shape))
    factors3_shape=[f.shape for f in factors3]
    Sum_of_variables_factors3=sum(list(x*y*z for x,y,z in factors3_shape))
    factors4_shape=[f.shape for f in factors4]
    Sum_of_variables_factors4=sum(list(x*y*z for x,y,z in factors4_shape))

    total_variabels=Sum_of_variables_factors1+Sum_of_variables_factors2+Sum_of_variables_factors3+Sum_of_variables_factors4

    CR_1=((total_variabels)+(d4*d5))/(d0*d1+d1*d2+d2*d3+d3*d4+d4*d5)
    CR_2=(total_variabels)/(d0*d1+d1*d2+d2*d3+d3*d4)
    print("CR_1 = "+str(CR_1)+"   "+"CR_2 = " +str(CR_2))


    results[Out_iter,0,:] = torch.tensor(loss1)
    results[Out_iter,1,:] = torch.tensor(loss2)
    results[Out_iter,2,:] = torch.tensor(accuracy_train)
    results[Out_iter,3,:] = torch.tensor(accuracy_test)
    results[Out_iter,4,:] = torch.tensor(time1)
    CR=(CR_1,CR_2)

#this postion to add new row into existing table
    #df=pd.read_csv('C:/Users/Mark/Desktop/rank40_100times.csv')
    #new_row = {'rank':rank, 'CR_1':CR_1, 'CR_2':CR_2, 'tau':tau, 'gamma':gamma, 'rho':rho, 'alpha':alpha, \
    #           'loss1':loss1[niter-1], 'loss2':loss2[niter-1], 'accuracy_train':accuracy_train[niter-1], \
    #           'accuracy_test':accuracy_test[niter-1], 'time':time1[niter-1], 'seed':seed} 
    #df=df.append(new_row,ignore_index=True)
    #df.to_csv('C:/Users/Mark/Desktop/rank40_100times.csv',index=False)


filename= "XavierUniform_" + "niter_"+ str(niter) + "rank_" + str(rank) + "tau_" + str(tau) + "gamma_" + str(gamma) + \
"rho_" + str(rho) + "alpha_" + str(alpha) + ".mat"
from scipy.io import savemat
%cd /home/c/cl237/TenBCD/UCI HAR/4 hidden layers/Different Initialization/
savemat (filename, {'results': torch.Tensor.numpy(results), 'Compression Ratio':CR})
#this position to save table into matlab
#df.to_csv('C:/Users/Mark/Desktop/result_compress_123_layer_highrank.csv')
##Changing Folder
 #%cd '/content/gdrive/MyDrive/Colab/Tensor-BCD-for-DNN/Experiments/DataSaved'
 #savemat("ThreeLayer_rank160.mat", {'results': torch.Tensor.numpy(results)})    

rank= 80 tau= 5 gamma= 0.5 rho= 0.5 alpha 1
Train on 7352 samples, validate on 2947 samples
Repeatition 1 Epoch 1 / 500 
 - time: 1.48870849609375 - sq_loss: 661.6954956054688 - tot_loss: 1023.9433062161843 - acc: 0.17995103373231774 - val_acc: 0.1866304716661011
Repeatition 1 Epoch 2 / 500 
 - time: 1.2653183937072754 - sq_loss: 294.0868225097656 - tot_loss: 475.9661811636761 - acc: 0.1868879216539717 - val_acc: 0.18052256532066507
Repeatition 1 Epoch 3 / 500 
 - time: 1.2622356414794922 - sq_loss: 160.82797241210938 - tot_loss: 261.37832728773355 - acc: 0.1868879216539717 - val_acc: 0.18052256532066507
Repeatition 1 Epoch 4 / 500 
 - time: 1.2444887161254883 - sq_loss: 87.1128158569336 - tot_loss: 147.4463349059224 - acc: 0.1868879216539717 - val_acc: 0.18052256532066507
Repeatition 1 Epoch 5 / 500 
 - time: 1.2362642288208008 - sq_loss: 46.864768981933594 - tot_loss: 85.90754327829927 - acc: 0.1868879216539717 - val_acc: 0.18052256532066507
Repeatition 1 Epoch 6 / 500 
 - time: 1.25

Repeatition 1 Epoch 48 / 500 
 - time: 1.229506492614746 - sq_loss: 0.0005266965017654002 - tot_loss: 0.9510751719626569 - acc: 0.8743199129488575 - val_acc: 0.8432304038004751
Repeatition 1 Epoch 49 / 500 
 - time: 1.234668254852295 - sq_loss: 0.0004912629956379533 - tot_loss: 0.9067427822392347 - acc: 0.8792165397170838 - val_acc: 0.8469630132337971
Repeatition 1 Epoch 50 / 500 
 - time: 1.2370450496673584 - sq_loss: 0.00045858966768719256 - tot_loss: 0.8702449737229472 - acc: 0.8824809575625681 - val_acc: 0.8506956226671191
Repeatition 1 Epoch 51 / 500 
 - time: 1.2300984859466553 - sq_loss: 0.00042841999675147235 - tot_loss: 0.8256994290986768 - acc: 0.8865614798694232 - val_acc: 0.8554462164913471
Repeatition 1 Epoch 52 / 500 
 - time: 1.2239627838134766 - sq_loss: 0.0004005375667475164 - tot_loss: 0.7950219858212222 - acc: 0.889417845484222 - val_acc: 0.8601968103155752
Repeatition 1 Epoch 53 / 500 
 - time: 1.2440094947814941 - sq_loss: 0.0003747817827388644 - tot_loss: 0.757995

Repeatition 1 Epoch 94 / 500 
 - time: 1.230912446975708 - sq_loss: 3.9809012378100306e-05 - tot_loss: 0.2136457551764579 - acc: 0.9446409140369967 - val_acc: 0.9284017645062775
Repeatition 1 Epoch 95 / 500 
 - time: 1.225926160812378 - sq_loss: 3.8170859625097364e-05 - tot_loss: 0.20955799668217878 - acc: 0.9455930359085963 - val_acc: 0.9304377332880895
Repeatition 1 Epoch 96 / 500 
 - time: 1.2400472164154053 - sq_loss: 3.6617904697777703e-05 - tot_loss: 0.20736087135526304 - acc: 0.9464091403699674 - val_acc: 0.9307770614183916
Repeatition 1 Epoch 97 / 500 
 - time: 1.2265515327453613 - sq_loss: 3.516402648529038e-05 - tot_loss: 0.20289951114364158 - acc: 0.9480413492927094 - val_acc: 0.9314557176789956
Repeatition 1 Epoch 98 / 500 
 - time: 1.2345366477966309 - sq_loss: 3.381115311640315e-05 - tot_loss: 0.19858189864589804 - acc: 0.948449401523395 - val_acc: 0.9334916864608076
Repeatition 1 Epoch 99 / 500 
 - time: 1.2267897129058838 - sq_loss: 3.2511394238099456e-05 - tot_loss: 0.

Repeatition 1 Epoch 140 / 500 
 - time: 1.2444467544555664 - sq_loss: 1.1652759894786868e-05 - tot_loss: 0.11450806616683451 - acc: 0.9715723612622416 - val_acc: 0.9474041398031897
Repeatition 1 Epoch 141 / 500 
 - time: 1.2304761409759521 - sq_loss: 1.1490313227113802e-05 - tot_loss: 0.11146313077928482 - acc: 0.9717083786724701 - val_acc: 0.9477434679334917
Repeatition 1 Epoch 142 / 500 
 - time: 1.2280206680297852 - sq_loss: 1.1336218449287117e-05 - tot_loss: 0.11373281137807112 - acc: 0.9719804134929271 - val_acc: 0.9484221241940957
Repeatition 1 Epoch 143 / 500 
 - time: 1.225017786026001 - sq_loss: 1.1186999472556636e-05 - tot_loss: 0.1140615238701912 - acc: 0.9723884657236126 - val_acc: 0.9487614523243977
Repeatition 1 Epoch 144 / 500 
 - time: 1.2234947681427002 - sq_loss: 1.104075363400625e-05 - tot_loss: 0.11380007563924721 - acc: 0.9726605005440696 - val_acc: 0.9491007804546997
Repeatition 1 Epoch 145 / 500 
 - time: 1.233715295791626 - sq_loss: 1.090690875571454e-05 - tot_l

Repeatition 1 Epoch 186 / 500 
 - time: 1.2253210544586182 - sq_loss: 7.88451507105492e-06 - tot_loss: 0.08659895380464633 - acc: 0.9798694232861807 - val_acc: 0.9545300305395318
Repeatition 1 Epoch 187 / 500 
 - time: 1.225210428237915 - sq_loss: 7.848397217458114e-06 - tot_loss: 0.08830163215559139 - acc: 0.9800054406964092 - val_acc: 0.9545300305395318
Repeatition 1 Epoch 188 / 500 
 - time: 1.2336430549621582 - sq_loss: 7.81031576480018e-06 - tot_loss: 0.09390944907036669 - acc: 0.9798694232861807 - val_acc: 0.9548693586698337
Repeatition 1 Epoch 189 / 500 
 - time: 1.227698564529419 - sq_loss: 7.773846846248489e-06 - tot_loss: 0.09328882117586801 - acc: 0.9800054406964092 - val_acc: 0.9545300305395318
Repeatition 1 Epoch 190 / 500 
 - time: 1.2331383228302002 - sq_loss: 7.732060112175532e-06 - tot_loss: 0.08724376901262332 - acc: 0.9800054406964092 - val_acc: 0.9545300305395318
Repeatition 1 Epoch 191 / 500 
 - time: 1.222416639328003 - sq_loss: 7.690699931117706e-06 - tot_loss: 0

...

Repeatition 1 Epoch 278 / 500 
 - time: 1.2362642288208008 - sq_loss: 5.726003564632265e-06 - tot_loss: 0.07616000438943615 - acc: 0.9862622415669206 - val_acc: 0.9589412962334578
Repeatition 1 Epoch 279 / 500 
 - time: 1.2368266582489014 - sq_loss: 5.7113179536827374e-06 - tot_loss: 0.07333442916467448 - acc: 0.986126224156692 - val_acc: 0.9589412962334578
Repeatition 1 Epoch 280 / 500 
 - time: 1.2170264720916748 - sq_loss: 5.695940672012512e-06 - tot_loss: 0.07419272447771164 - acc: 0.986126224156692 - val_acc: 0.9589412962334578
Repeatition 1 Epoch 281 / 500 
 - time: 1.2378497123718262 - sq_loss: 5.680429239873774e-06 - tot_loss: 0.07290497608968849 - acc: 0.9859902067464635 - val_acc: 0.9589412962334578
Repeatition 1 Epoch 282 / 500 
 - time: 1.236699104309082 - sq_loss: 5.666747256327653e-06 - tot_loss: 0.07266517939805439 - acc: 0.986126224156692 - val_acc: 0.9582626399728538
Repeatition 1 Epoch 283 / 500 
 - time: 1.2182645797729492 - sq_loss: 5.650004823110066e-06 - tot_loss:

Repeatition 1 Epoch 324 / 500 
 - time: 1.2301123142242432 - sq_loss: 5.125672942085657e-06 - tot_loss: 0.07398223096248735 - acc: 0.9878944504896626 - val_acc: 0.9602986087546658
Repeatition 1 Epoch 325 / 500 
 - time: 1.246960163116455 - sq_loss: 5.116426109452732e-06 - tot_loss: 0.07450805084256551 - acc: 0.9880304678998912 - val_acc: 0.9602986087546658
Repeatition 1 Epoch 326 / 500 
 - time: 1.2335853576660156 - sq_loss: 5.105210220790468e-06 - tot_loss: 0.06955515412327173 - acc: 0.9880304678998912 - val_acc: 0.9602986087546658
Repeatition 1 Epoch 327 / 500 
 - time: 1.2283406257629395 - sq_loss: 5.09200162923662e-06 - tot_loss: 0.07057095743021513 - acc: 0.9880304678998912 - val_acc: 0.9602986087546658
Repeatition 1 Epoch 328 / 500 
 - time: 1.2306890487670898 - sq_loss: 5.081744802737376e-06 - tot_loss: 0.07233495017769265 - acc: 0.9881664853101197 - val_acc: 0.9602986087546658
Repeatition 1 Epoch 329 / 500 
 - time: 1.2303760051727295 - sq_loss: 5.071027771919034e-06 - tot_loss

Repeatition 1 Epoch 370 / 500 
 - time: 1.2279503345489502 - sq_loss: 4.649939000955783e-06 - tot_loss: 0.07157157461708685 - acc: 0.9887105549510338 - val_acc: 0.9599592806243638
Repeatition 1 Epoch 371 / 500 
 - time: 1.2364115715026855 - sq_loss: 4.6420018406934105e-06 - tot_loss: 0.07890465863733276 - acc: 0.9887105549510338 - val_acc: 0.9599592806243638
Repeatition 1 Epoch 372 / 500 
 - time: 1.226186752319336 - sq_loss: 4.632435320672812e-06 - tot_loss: 0.07263476909608002 - acc: 0.9887105549510338 - val_acc: 0.9599592806243638
Repeatition 1 Epoch 373 / 500 
 - time: 1.2370696067810059 - sq_loss: 4.624668235919671e-06 - tot_loss: 0.06854007620228764 - acc: 0.9887105549510338 - val_acc: 0.9599592806243638
Repeatition 1 Epoch 374 / 500 
 - time: 1.2354519367218018 - sq_loss: 4.617805188900093e-06 - tot_loss: 0.06858293985491848 - acc: 0.9887105549510338 - val_acc: 0.9599592806243638
Repeatition 1 Epoch 375 / 500 
 - time: 1.2360937595367432 - sq_loss: 4.610893029166618e-06 - tot_lo

Repeatition 1 Epoch 416 / 500 
 - time: 1.232377290725708 - sq_loss: 4.28333441959694e-06 - tot_loss: 0.0661136376691207 - acc: 0.9891186071817193 - val_acc: 0.9623345775364778
Repeatition 1 Epoch 417 / 500 
 - time: 1.2196357250213623 - sq_loss: 4.276167146599619e-06 - tot_loss: 0.0713358755817719 - acc: 0.9892546245919478 - val_acc: 0.9626739056667798
Repeatition 1 Epoch 418 / 500 
 - time: 1.2392079830169678 - sq_loss: 4.268322754796827e-06 - tot_loss: 0.06807492965224426 - acc: 0.9891186071817193 - val_acc: 0.9630132337970818
Repeatition 1 Epoch 419 / 500 
 - time: 1.2195594310760498 - sq_loss: 4.259314209775766e-06 - tot_loss: 0.06993012080353012 - acc: 0.9891186071817193 - val_acc: 0.9633525619273838
Repeatition 1 Epoch 420 / 500 
 - time: 1.2185957431793213 - sq_loss: 4.251588052284205e-06 - tot_loss: 0.06699194223761396 - acc: 0.9892546245919478 - val_acc: 0.9626739056667798
Repeatition 1 Epoch 421 / 500 
 - time: 1.2317993640899658 - sq_loss: 4.244392584951129e-06 - tot_loss: 

Repeatition 1 Epoch 462 / 500 
 - time: 1.2523934841156006 - sq_loss: 3.9732140066917054e-06 - tot_loss: 0.07018287624613428 - acc: 0.9895266594124048 - val_acc: 0.9640312181879878
Repeatition 1 Epoch 463 / 500 
 - time: 1.2538316249847412 - sq_loss: 3.966154508816544e-06 - tot_loss: 0.06923038360159417 - acc: 0.9895266594124048 - val_acc: 0.9640312181879878
Repeatition 1 Epoch 464 / 500 
 - time: 1.249070644378662 - sq_loss: 3.9600604395673145e-06 - tot_loss: 0.07235206349589696 - acc: 0.9895266594124048 - val_acc: 0.9640312181879878
Repeatition 1 Epoch 465 / 500 
 - time: 1.2500474452972412 - sq_loss: 3.954189196520019e-06 - tot_loss: 0.06416270422792536 - acc: 0.9895266594124048 - val_acc: 0.9640312181879878
Repeatition 1 Epoch 466 / 500 
 - time: 1.2504703998565674 - sq_loss: 3.948240191675723e-06 - tot_loss: 0.06729071131825037 - acc: 0.9896626768226333 - val_acc: 0.9643705463182898
Repeatition 1 Epoch 467 / 500 
 - time: 1.2466683387756348 - sq_loss: 3.942018338420894e-06 - tot_l